In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta, date
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
import pickle

from multiprocessing import Pool
from functools import partial
import Training_function as func

In [2]:
data = pd.read_csv('D:\\庫存健診開發\\data\\Training\\Raw\\large_noisy.csv', converters={'ts': str, 'StockNo': str, 'StockName': str})
data['ts'] = pd.to_datetime(data['ts'])
data

,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,...,SOk5,SOd3,average_volatility,index_slope,industry_slope,origin_VWAP,origin_ATR,signal,MACD_diff,cluster
0,2007-07-02,1452,宏益,9.30,9.33,9.25,9.28,-0.32,0.86,1115.0,...,0.000000,0.000000,NaN,-0.95,-1.84,9.291480,NaN,NaN,NaN,0
1,2007-07-03,1452,宏益,9.35,9.38,9.28,9.31,0.32,1.08,2810.0,...,0.000000,0.000000,NaN,-0.31,-1.13,9.335587,NaN,NaN,NaN,0
2,2007-07-04,1452,宏益,9.45,9.96,9.40,9.96,6.98,6.02,7034.0,...,0.000000,0.000000,NaN,6.17,4.43,9.753767,NaN,NaN,NaN,0
3,2007-07-05,1452,宏益,9.72,10.15,9.41,9.89,2.38,7.66,9277.0,...,0.000000,0.000000,NaN,1.50,-0.74,9.812332,NaN,NaN,NaN,0
4,2007-07-06,1452,宏益,9.90,9.92,9.60,9.67,-2.22,3.24,2005.0,...,0.553477,0.285666,NaN,-2.65,-5.02,9.748130,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398478,2019-09-17,9958,世紀鋼,78.90,78.90,77.10,77.30,-0.26,2.32,3050.0,...,0.564237,0.660063,2.848326,-0.04,0.88,77.767541,2.215357,0.926006,1.576201,4
1398479,2019-09-18,9958,世紀鋼,77.90,79.30,77.50,77.70,0.91,2.34,5940.0,...,0.582582,0.621322,2.784785,0.40,0.66,78.262458,2.163437,1.123856,1.384947,4
1398480,2019-09-19,9958,世紀鋼,78.40,78.40,76.20,76.30,-1.80,2.83,4011.0,...,0.173262,0.397292,2.790437,-1.48,-1.59,76.962354,2.168007,1.248822,0.874766,4
1398481,2019-09-20,9958,世紀鋼,76.60,77.00,75.80,75.80,-0.66,1.57,1742.0,...,0.100148,0.248720,2.637883,-0.98,-1.16,76.280712,2.047007,1.300410,0.361115,4


In [3]:
data['sd_15'] = data['sd_15']/data['ma_15']

In [4]:
train_df, val_df, test_df = func.TrainTestSplit(data, date(2016,1,1), date(2017,9,1))
del data

D:\庫存健診開發\code\TrainingPrep\Training_function.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_df = data[data['ts'].dt.date < TrainDate][data['pvt'].notnull()][data['MACD_diff'].notnull()]
D:\庫存健診開發\code\TrainingPrep\Training_function.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_df = data[data['ts'].dt.date < valDate][data['ts'].dt.date >= TrainDate][data['pvt'].notnull()][data['MACD_diff'].notnull()]
D:\庫存健診開發\code\TrainingPrep\Training_function.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_df = data[data['ts'].dt.date >= valDate][data['pvt'].notnull()][data['MACD_diff'].notnull()]


In [5]:
feature_list = ['return', 'volatility', 'vol', 'total', 'capital', 'VWAP', 
       'industry', 
       'index_return', 'index_vol', 'industry_return', 'industry_vol',
       'foreign_buy', 'investment_buy', 'dealer_buy', 'MACD', 'sd_15', 'foreign_buy_ratio', 'foreign_buy_week_sum',
       'foreign_buy_2week_sum', 'foreign_buy_3week_sum',
       'foreign_buy_continuous', 'investment_buy_ratio',
       'investment_buy_week_sum', 'investment_buy_2week_sum',
       'investment_buy_3week_sum', 'investment_buy_continuous',
       'dealer_buy_ratio', 'dealer_buy_week_sum', 'dealer_buy_2week_sum',
       'dealer_buy_3week_sum', 'dealer_buy_continuous', 'VWAP_momentum_day1',
       'index_momentum_day1', 'industry_momentum_day1', 'VWAP_momentum_day5',
       'index_momentum_day5', 'industry_momentum_day5', 'VWAP_momentum_day15',
       'index_momentum_day15', 'industry_momentum_day15', 'index_close_corr',
       'industry_close_corr', 'VWAP_skew', 'VWAP_kurtosis', 'index_close_skew',
       'index_close_kurtosis', 'industry_close_skew',
       'industry_close_kurtosis', 'foreign_buy_skew', 'foreign_buy_kurtosis',
       'investment_buy_skew', 'investment_buy_kurtosis', 'dealer_buy_skew',
       'dealer_buy_kurtosis', 'vol_skew', 'vol_kurtosis', 'index_pvt',
       'industry_pvt', 'foreign_pvt', 'investment_pvt', 'dealer_pvt', 'pvt',
       'ATR', 'RSI_15', 'SOk5', 'SOd3', 'average_volatility', 'index_slope',
       'industry_slope', 'MACD_diff']

industry_index = ['industry_open', 'industry_high', 'industry_low', 'industry_close', 'industry_vol', 
                  'industry_pvt', 'industry_close_skew', 'industry_close_kurtosis']

print(len(feature_list))
print(train_df[feature_list].isna().values.any())
print(val_df[feature_list].isna().values.any())
print(test_df[feature_list].isna().values.any())

70
False
False
False


In [6]:
# param, industry_param = func.GetScalerParam(train_df, feature_list, industry_index)
# param

,max,min,std,mean
return,1.000000e+01,-1.007000e+01,2.428825e+00,2.958703e-02
volatility,1.992000e+01,0.000000e+00,1.834412e+00,2.873412e+00
vol,6.354610e+05,0.000000e+00,1.155645e+04,5.265181e+03
total,2.768190e+07,0.000000e+00,5.215300e+05,1.977153e+05
capital,4.006140e+04,1.100000e+00,1.509922e+03,4.486371e+02
VWAP,3.690686e+03,0.000000e+00,8.532014e+01,4.671414e+01
index_open,9.998080e+03,3.962690e+03,1.131070e+03,7.997704e+03
index_high,1.001428e+04,4.172490e+03,1.125459e+03,8.035784e+03
index_low,9.937040e+03,3.955430e+03,1.133816e+03,7.944492e+03
index_close,9.973120e+03,4.089930e+03,1.128220e+03,7.988257e+03


In [6]:
# train_df, val_df, test_df = func.Scaler(train_df, param, industry_param, feature_list, industry_index), func.Scaler(val_df, param, industry_param, feature_list, industry_index), func.Scaler(test_df, param, industry_param, feature_list, industry_index)
# display(train_df)
# display(test_df)

D:\庫存健診開發\code\TrainingPrep\Training_function.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{col}_scaled'] = (df[col] - industry_min.loc[col])/(industry_max.loc[col] - industry_min.loc[col])
D:\庫存健診開發\code\TrainingPrep\Training_function.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = (df[feature] - Min.loc[feature])/(Max.loc[feature] - Min.loc[feature])


,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,...,origin_VWAP,origin_ATR,industry_open_scaled,industry_high_scaled,industry_low_scaled,industry_close_scaled,industry_vol_scaled,industry_pvt_scaled,industry_close_skew_scaled,industry_close_kurtosis_scaled
20,2007-07-30,1452,宏益,10.75,10.90,10.50,10.70,0.478326,0.186747,0.001438,...,10.723195,0.747903,0.568429,0.558011,0.563381,0.556493,0.322765,0.836447,0.514569,0.007818
21,2007-07-31,1452,宏益,11.00,11.20,10.90,10.90,0.594918,0.140562,0.001349,...,11.033839,0.712120,0.586465,0.577344,0.580704,0.575440,0.289363,0.834986,0.509863,0.017115
22,2007-08-01,1452,宏益,10.95,11.40,10.80,10.90,0.501744,0.276104,0.002214,...,11.095238,0.697424,0.575707,0.582040,0.556189,0.540590,0.380331,0.608718,0.466669,0.063933
23,2007-08-02,1452,宏益,11.10,11.30,10.15,10.70,0.410563,0.529618,0.001457,...,10.818575,0.756388,0.566045,0.554917,0.523510,0.526089,0.328502,0.426497,0.406377,0.120815
24,2007-08-03,1452,宏益,11.20,11.20,10.85,10.85,0.571500,0.164157,0.000834,...,10.975472,0.707792,0.553334,0.546183,0.555736,0.545277,0.282107,0.368129,0.427442,0.079052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909089,2015-12-25,1605,華新,7.41,7.53,7.41,7.50,0.548580,0.081325,0.010383,...,7.490300,0.151178,0.150272,0.131731,0.151974,0.138738,0.020127,0.539121,0.503397,0.100656
909090,2015-12-28,1605,華新,7.50,7.54,7.43,7.44,0.461883,0.073795,0.007414,...,7.488219,0.146031,0.153621,0.131731,0.154963,0.139822,0.012169,0.543043,0.514767,0.081856
909091,2015-12-29,1605,華新,7.47,7.55,7.42,7.45,0.508221,0.087851,0.008169,...,7.486997,0.144027,0.154667,0.132804,0.154322,0.137221,0.014763,0.541602,0.485234,0.057336
909092,2015-12-30,1605,華新,7.53,7.53,7.45,7.45,0.501744,0.053715,0.007481,...,7.481910,0.136023,0.152156,0.133019,0.154536,0.136354,0.011385,0.541299,0.435652,0.066040


,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,...,origin_VWAP,origin_ATR,industry_open_scaled,industry_high_scaled,industry_low_scaled,industry_close_scaled,industry_vol_scaled,industry_pvt_scaled,industry_close_skew_scaled,industry_close_kurtosis_scaled
1124829,2017-09-01,1439,中和,21.3,21.30,20.70,21.0,0.525660,0.144076,0.000087,...,21.163636,0.616250,0.725555,0.727183,0.731787,0.726715,0.103402,0.408331,0.456492,0.114351
1124830,2017-09-04,1439,中和,20.8,21.35,20.75,21.0,0.501744,0.143574,0.000052,...,21.121212,0.614218,0.733070,0.723035,0.732348,0.720779,0.084984,0.402067,0.458254,0.099217
1124831,2017-09-05,1439,中和,20.8,21.00,20.80,20.8,0.454410,0.047691,0.000047,...,21.200000,0.577593,0.726242,0.721477,0.732564,0.722597,0.075081,0.404334,0.424450,0.130042
1124832,2017-09-06,1439,中和,20.8,21.00,20.70,20.7,0.477828,0.072289,0.000118,...,20.880000,0.567894,0.729355,0.719239,0.717488,0.705314,0.076368,0.394406,0.494141,0.048594
1124833,2017-09-07,1439,中和,20.7,20.70,20.15,20.2,0.381166,0.133534,0.000195,...,20.524194,0.588157,0.712221,0.714982,0.718460,0.708731,0.078176,0.405296,0.499333,0.039078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333480,2019-09-17,6505,台塑化,105.0,105.50,102.00,102.0,0.406079,0.169177,0.011165,...,103.390839,2.602013,0.958564,0.926083,0.947931,0.906364,0.051036,0.672806,0.855980,0.523593
1333481,2019-09-18,6505,台塑化,100.5,103.00,98.60,101.5,0.477329,0.216365,0.012221,...,100.724440,2.875616,0.880070,0.884751,0.886983,0.898937,0.069216,0.669831,0.760111,0.307729
1333482,2019-09-19,6505,台塑化,100.5,101.00,99.70,99.7,0.413553,0.064257,0.004661,...,100.098582,2.678664,0.881383,0.851883,0.907750,0.867919,0.053126,0.655073,0.729470,0.261354
1333483,2019-09-20,6505,台塑化,99.7,101.00,99.50,101.0,0.566517,0.075301,0.007151,...,100.357835,2.531331,0.867669,0.851883,0.905342,0.890928,0.069450,0.669167,0.696658,0.179970


In [6]:
buy_feature = ['return', 'vol', 'total', 'capital', 'VWAP', 
       'foreign_buy', 'investment_buy', 'dealer_buy', 'foreign_buy_ratio', 'foreign_buy_week_sum',
       'foreign_buy_2week_sum', 'foreign_buy_3week_sum',
       'foreign_buy_continuous', 'investment_buy_ratio',
       'investment_buy_week_sum', 'investment_buy_2week_sum',
       'investment_buy_3week_sum', 'investment_buy_continuous',
       'dealer_buy_ratio', 'dealer_buy_week_sum', 'dealer_buy_2week_sum',
       'dealer_buy_3week_sum', 'dealer_buy_continuous', 'foreign_buy_skew', 'foreign_buy_kurtosis',
       'investment_buy_skew', 'investment_buy_kurtosis', 'dealer_buy_skew',
       'dealer_buy_kurtosis', 'foreign_pvt', 'investment_pvt', 'dealer_pvt', 'ATR']

index_feature =  ['return',
       'vol', 'total', 'capital', 'VWAP', 
       'index_return', 'index_vol', 'industry_return', 'industry_vol',
       'index_momentum_day1', 'industry_momentum_day1', 
       'index_momentum_day5', 'industry_momentum_day5', 
       'index_momentum_day15', 'industry_momentum_day15', 'index_close_skew',
       'index_close_kurtosis', 'industry_close_skew',
       'industry_close_kurtosis', 'index_pvt',
       'industry_pvt', 'index_slope',
       'industry_slope', 'ATR']

technical_feature = ['return',
       'volatility', 'vol', 'total', 'capital', 'VWAP', 
       'ma_15', 'ema_20',
       'ema_5', 'MACD', 'sd_15','VWAP_momentum_day1',
      'VWAP_momentum_day5', 'VWAP_momentum_day15',
       'VWAP_skew', 'VWAP_kurtosis', 'vol_skew', 'vol_kurtosis', 'pvt',
       'ATR', 'RSI_15', 'SOk5', 'SOd3', 'average_volatility', 'MACD_diff']

In [7]:
df_list = [group[1] for group in train_df.groupby(train_df['StockNo'])]

X_train, Y_train = [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.separate_elimination, lookback=15, forward=5, feature_list=buy_feature), df_list), total=len(df_list)), 1):
            for xx in x[0]:
                X_train.append(xx.tolist())
            Y_train.extend(x[1])
            

assert len(X_train) == len(Y_train)
save_list = [X_train, Y_train]
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\DL\\Whole_classification_minmax0_weekly_train_buy', 'wb') as fp:
    pickle.dump(save_list, fp)

In [8]:
df_list = [group[1] for group in val_df.groupby(val_df['StockNo'])]

X_val, Y_val = [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.separate_elimination, lookback=15, forward=5, feature_list=buy_feature), df_list), total=len(df_list)), 1):
            for xx in x[0]:
                X_val.append(xx.tolist())
            Y_val.extend(x[1])

assert len(X_val) == len(Y_val)
save_list = [X_val, Y_val]
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\DL\\Whole_classification_minmax0_weekly_val_buy', 'wb') as fp:
    pickle.dump(save_list, fp)

In [11]:
df_list = [group[1] for group in test_df.groupby(test_df['StockNo'])]

X_test, Y_test = [], []

if __name__ == '__main__':
    with Pool(processes=10) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.create_dataset, lookback=15, forward=5, feature_list=buy_feature, problem='classification'), df_list), total=len(df_list)), 1):
            for xx in x[0]:
                X_test.append(xx.tolist())
            Y_test.extend(x[1])

assert len(X_test) == len(Y_test)
save_list = [X_test, Y_test]
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\DL\\Whole_classification_minmax0_weekly_test_buy', 'wb') as fp:
    pickle.dump(save_list, fp)

In [8]:
# with open('D:\\庫存健診開發\\data\\Training\\processed\\DL\\Whole_classification_minmax0_weekly_train_buy', 'rb') as fp:
#     load_list = pickle.load(fp)
    
# X_train, Y_train = load_list[0], load_list[1]
    
train_data = func.transform_whole_csv(X_train, Y_train, buy_feature, lookback=15)
#train_data = pd.merge(train_data, train_df[['ts', 'StockNo', 'cluster']], on=['ts', 'StockNo'], how='left')
train_data

,return_day1,vol_day1,total_day1,capital_day1,VWAP_day1,foreign_buy_day1,investment_buy_day1,dealer_buy_day1,foreign_buy_ratio_day1,foreign_buy_week_sum_day1,...,dealer_buy_kurtosis_day15,foreign_pvt_day15,investment_pvt_day15,dealer_pvt_day15,ATR_day15,Y,ts,StockNo,cluster,industry
0,2.39,28881.0,1234551.0,1390.4,42.746131,-738.0,6447.0,-289.0,-0.025553,-15407.0,...,-0.447285,4003.334729,1316.754803,-63.117697,2.077990,-1,2007-09-03,1101,3,水泥工業
1,-6.89,18071.0,731497.0,1294.6,40.479055,-6817.0,409.0,94.0,-0.377234,-22266.0,...,-0.543822,2826.223285,801.074997,26.156479,2.180935,0,2007-09-04,1101,3,水泥工業
2,-6.90,30715.0,1146931.0,1205.3,37.341071,-14830.0,-233.0,-13.0,-0.482826,-31793.0,...,-0.580919,2890.287767,800.105079,29.214249,2.139500,0,2007-09-05,1101,3,水泥工業
3,-2.96,36804.0,1314173.0,1169.5,35.707342,-12658.0,37.0,-465.0,-0.343930,-35681.0,...,-0.663385,2898.631866,853.568876,14.005412,2.128296,0,2007-09-06,1101,3,水泥工業
4,5.00,24456.0,923621.0,1228.0,37.766642,-12293.0,-461.0,229.0,-0.502658,-47336.0,...,-0.494869,2454.344036,913.476716,68.876520,1.999750,0,2007-09-07,1101,3,水泥工業
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887222,-0.93,306.0,22805.0,199.7,74.526144,-8.0,0.0,-5.0,-0.026144,-102.0,...,9.478183,-0.010023,0.261826,-0.483191,0.979190,0,2015-12-17,9941,2,其他
887223,-0.54,381.0,27948.0,198.7,73.354331,-18.0,-20.0,-1.0,-0.047244,-136.0,...,7.175011,0.018541,0.222170,-0.464517,0.928199,0,2015-12-18,9941,2,其他
887224,-0.54,383.0,28139.0,197.6,73.469974,-11.0,-97.0,0.0,-0.028721,-125.0,...,7.077585,-0.027531,0.543607,-0.420477,0.931596,0,2015-12-21,9941,2,其他
887225,-0.41,243.0,17960.0,196.8,73.909465,-29.0,0.0,0.0,-0.119342,-98.0,...,7.077585,-0.088651,0.506143,-0.420477,0.840146,0,2015-12-22,9941,2,其他


In [10]:
train_data1 = train_data.iloc[:450000]
train_data2 = train_data.iloc[450000:]

In [9]:
train_data.to_csv(f'D:\\OneDrive - SinoPac\\training\\train_large_buy.csv', index=False)

In [9]:
val_data = func.transform_whole_csv(X_val, Y_val, buy_feature, lookback=15)
#val_data= pd.merge(val_data, val_df[['ts', 'StockNo', 'cluster']], on = ['ts', 'StockNo'], how='left')
val_data

,return_day1,vol_day1,total_day1,capital_day1,VWAP_day1,foreign_buy_day1,investment_buy_day1,dealer_buy_day1,foreign_buy_ratio_day1,foreign_buy_week_sum_day1,...,dealer_buy_kurtosis_day15,foreign_pvt_day15,investment_pvt_day15,dealer_pvt_day15,ATR_day15,Y,ts,StockNo,cluster,industry
0,0.24,1504.0,30971.0,152.8,20.592420,-14.0,0.0,-16.0,-0.009309,659.0,...,1.530903,6.545513,-0.224491,19.221513,0.396718,0,2016-01-22,1210,2,食品工業
1,-2.89,1046.0,21357.0,148.4,20.417782,148.0,0.0,1.0,0.141491,684.0,...,1.519792,6.187864,-0.224491,19.278723,0.372129,0,2016-01-25,1210,2,食品工業
2,-0.50,1092.0,22017.0,147.6,20.162088,404.0,0.0,-3.0,0.369963,884.0,...,0.964765,3.539640,0.232315,17.798218,0.363113,0,2016-01-26,1210,2,食品工業
3,-1.75,1276.0,25412.0,145.1,19.915361,77.0,0.0,75.0,0.060345,806.0,...,0.963235,7.238894,-0.092434,16.147408,0.336474,0,2016-01-27,1210,2,食品工業
4,3.05,1128.0,22771.0,149.5,20.187057,203.0,0.0,38.0,0.179965,818.0,...,1.034907,6.095124,-0.092434,14.403604,0.313164,0,2016-01-28,1210,2,食品工業
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198758,0.15,1143.0,37623.0,458.4,32.916010,-242.0,0.0,15.0,-0.211724,-3457.0,...,5.535053,25.673422,-1.004505,7.053181,0.479178,1,2017-08-17,9945,3,其他
198759,-0.61,2429.0,79830.0,455.6,32.865377,-1057.0,0.0,12.0,-0.435159,-3844.0,...,5.134717,34.846220,-0.044559,6.340269,0.458575,1,2017-08-18,9945,3,其他
198760,1.53,2671.0,88338.0,462.6,33.073006,310.0,-8.0,20.0,0.116061,-2588.0,...,4.603683,29.432080,-0.044559,7.074364,0.476254,1,2017-08-21,9945,3,其他
198761,-0.15,861.0,28543.0,461.9,33.150987,16.0,0.0,-10.0,0.018583,-1534.0,...,2.687513,28.216000,-0.044559,9.985950,0.518744,1,2017-08-22,9945,3,其他


In [10]:
val_data.to_csv(f'D:\\OneDrive - SinoPac\\training\\val_large_buy.csv', index=False)

In [12]:
test_data = func.transform_whole_csv(X_test, Y_test, buy_feature, lookback=15)
#test_data= pd.merge(test_data, test_df[['ts', 'StockNo', 'cluster']], on = ['ts', 'StockNo'], how='left')
test_data

,return_day1,vol_day1,total_day1,capital_day1,VWAP_day1,foreign_buy_day1,investment_buy_day1,dealer_buy_day1,foreign_buy_ratio_day1,foreign_buy_week_sum_day1,...,dealer_buy_kurtosis_day15,foreign_pvt_day15,investment_pvt_day15,dealer_pvt_day15,ATR_day15,Y,ts,StockNo,cluster,industry
0,0.56,1773.0,31857.0,63.1,17.967851,318.0,0.0,0.0,0.179357,695.0,...,5.402733,-4.715792,1.598721e-14,-0.108582,0.373438,1,2017-09-21,1218,1,食品工業
1,0.28,1245.0,22351.0,63.2,17.952610,-168.0,0.0,0.0,-0.134940,378.0,...,5.284030,-2.182869,1.598721e-14,-0.147332,0.477170,0,2017-09-22,1218,1,食品工業
2,-0.84,814.0,14528.0,62.7,17.847666,33.0,0.0,2.0,0.040541,563.0,...,5.284030,-3.816454,1.598721e-14,-0.147332,0.486274,0,2017-09-25,1218,1,食品工業
3,0.56,1214.0,21795.0,63.1,17.953048,-212.0,0.0,0.0,-0.174629,161.0,...,5.284030,-10.430459,1.598721e-14,-0.117479,0.512990,0,2017-09-26,1218,1,食品工業
4,-1.68,1352.0,24021.0,62.0,17.767012,40.0,0.0,-1.0,0.029586,11.0,...,5.284030,-6.321865,1.598721e-14,-0.117479,0.530036,0,2017-09-27,1218,1,食品工業
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261392,-0.82,1282.0,94273.0,152.6,73.535881,-173.0,0.0,-18.0,-0.134945,-324.0,...,1.599542,20.225802,3.053559e+01,39.272030,2.227654,1,2019-09-06,9958,4,鋼鐵工業
261393,-1.10,1406.0,101935.0,150.9,72.500000,-9.0,-330.0,28.0,-0.006401,-160.0,...,1.121826,20.186656,3.053559e+01,38.064468,2.174197,0,2019-09-09,9958,4,鋼鐵工業
261394,1.25,1222.0,88963.0,152.8,72.801146,12.0,-25.0,72.0,0.009820,63.0,...,1.193529,16.244066,3.053559e+01,38.027537,2.164922,1,2019-09-10,9958,4,鋼鐵工業
261395,-2.33,3202.0,230153.0,149.2,71.877889,-339.0,-615.0,-160.0,-0.105871,-244.0,...,0.012374,21.734884,3.053559e+01,44.403342,2.169307,0,2019-09-11,9958,4,鋼鐵工業


In [13]:
test_data.to_csv(f'D:\\OneDrive - SinoPac\\training\\test_large_buy.csv', index=False)

In [ ]:
len(test_data[test_data.Y == 1])/len(test_data)